#IMDB Review Sentiment

Library

In [1]:
# !python -m spacy download en_core_web_lg

In [2]:
import pandas as pd # data frame
import numpy as np # array processing
from nltk.tokenize import word_tokenize # Tokenizing words
from nltk.corpus import stopwords  # stopwords
from nltk.stem import WordNetLemmatizer  #Lemmetization
lem = WordNetLemmatizer() #Lemmetization object
from sklearn.model_selection import train_test_split # Test train split
from sklearn.linear_model import LogisticRegression # classification model feature Base methord
from sklearn.svm import LinearSVC #Classification model feature Base Methord
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix # Verification metrics
from wordcloud import WordCloud # wordcloud
from sklearn.feature_extraction.text import TfidfVectorizer #word to vector
import matplotlib.pyplot as plt #plot
import seaborn as sns #plot 
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stopWords = set(stopwords.words('english')) #Downloading Stopwords in stopWords
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df=pd.read_csv('/content/drive/MyDrive/imdb_reviews.txt',sep='\t',header=None)
df.head(2)

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0


#Text Preprocessing

In [5]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and ",
"won't": "would not",
"wouldn't": "would not",
'dis': 'this',
'bak': 'back',
'brng': 'bring'}

In [6]:
def cont_to_exp(x):
  if x in contractions.keys():
    value=contractions[x]
  else:
    value=x
  return value

In [7]:
def preprocess(x):

  x = x.lower()
  x = ' '.join(list((cont_to_exp(t) for t in x.split())))
  x = re.sub('[^a-zA-Z]', ' ',x)
  x = re.sub('(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','', x)
  x = re.sub('([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)','',x)
  x = re.sub('[\W+]', ' ', x)
  x = ' '.join(list((lem.lemmatize(t) for t in x.split() if t not in stopWords)))
  x = ' '.join(list((t for t in x.split() if len(t)>1)))
  return x

In [8]:
df['pro_text']=df[0].apply(lambda x :  preprocess(x))

In [9]:
X1=df['pro_text']
y1=df[1]

#TFIDF Vectoriser

In [10]:
tf=TfidfVectorizer(max_features=1240)
text_tf= tf.fit_transform(X1)

In [11]:
X, X_test, y, y_test = train_test_split(text_tf, y1, test_size=0.2, random_state=0)

#Support Vector Classifier

In [12]:
lsvc=LinearSVC()
hyperparameters={'penalty':['l1','l2'],'C':(1,2,3,4)}
svc=GridSearchCV(lsvc,hyperparameters,n_jobs=-1,cv=5)
svc.fit(X,y)
y_pred_svc = svc.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "/usr/loca

In [13]:
svc.best_params_

{'C': 1, 'penalty': 'l2'}

In [14]:
print(classification_report(y_test,y_pred_svc))

              precision    recall  f1-score   support

           0       0.87      0.70      0.78        84
           1       0.70      0.86      0.77        66

    accuracy                           0.77       150
   macro avg       0.78      0.78      0.77       150
weighted avg       0.79      0.77      0.77       150



#Logistic Regression

In [15]:
LR=LogisticRegression(solver='liblinear')
hyperparameters={'penalty':['l1','l2'],'C':(1,2,3,4)}
lr=GridSearchCV(LR,hyperparameters,n_jobs=-1,cv=5)
lr.fit(X,y)
y_pred_lr=lr.predict(X_test)#0.71

In [16]:
lr.best_params_

{'C': 4, 'penalty': 'l2'}

In [17]:
print(classification_report(y_test,y_pred_lr))

              precision    recall  f1-score   support

           0       0.88      0.67      0.76        84
           1       0.67      0.88      0.76        66

    accuracy                           0.76       150
   macro avg       0.77      0.77      0.76       150
weighted avg       0.79      0.76      0.76       150



#Random Forest Classifier

In [18]:
# RFC = RandomForestClassifier(criterion='entropy', n_estimators=200).fit(X, y)
RFC= RandomForestClassifier()
hyperparameters={'n_estimators': [100,200,300,400,500],'criterion' :['gini', 'entropy']}
rfc=GridSearchCV(RFC,hyperparameters,n_jobs=-1,cv=5)
rfc.fit(X,y)
y_pred_rfc=rfc.predict(X_test)#0.63

In [19]:
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.78      0.54      0.63        84
           1       0.58      0.80      0.67        66

    accuracy                           0.65       150
   macro avg       0.68      0.67      0.65       150
weighted avg       0.69      0.65      0.65       150



In [20]:
rfc.best_params_

{'criterion': 'entropy', 'n_estimators': 300}

In [21]:
df

,0,1,pro_text
0,"A very, very, very slow-moving, aimless movie ...",0,slow moving aimless movie distressed drifting ...
1,Not sure who was more lost - the flat characte...,0,sure lost flat character audience nearly half ...
2,Attempting artiness with black & white and cle...,0,attempting artiness black white clever camera ...
3,Very little music or anything to speak of.,0,little music anything speak
4,The best scene in the movie was when Gerardo i...,1,best scene movie gerardo trying find song keep...
...,...,...,...
743,I just got bored watching Jessice Lange take h...,0,got bored watching jessice lange take clothes
744,"Unfortunately, any virtue in this film's produ...",0,unfortunately virtue film production work lost...
745,"In a word, it is embarrassing.",0,word embarrassing
746,Exceptionally bad!,0,exceptionally bad


# Word2Vec

In [22]:
import spacy

In [23]:
# spacy.cli.download("en_core_web_lg")
nlp=spacy.load('en_core_web_lg')

In [24]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [25]:
df['vec']=df[0].apply(lambda x : get_vec(str(x)))

To increase the process speed 

In [26]:
X=df['vec'].to_numpy()
X.shape

(748,)

reshape to one column and multiple rows

In [27]:
X=X.reshape(-1,1)
X.shape

(748, 1)

In [28]:
X=np.concatenate(np.concatenate(X,axis=0),axis=0).reshape(-1,300)
X

array([[-0.0714336 ,  0.2527413 , -0.2089631 , ..., -0.06899667,
        -0.2158687 ,  0.02880419],
       [ 0.05112227,  0.21386255, -0.13622889, ..., -0.02658024,
         0.08020092, -0.05226086],
       [-0.17059676,  0.07404754, -0.1405276 , ..., -0.07198143,
         0.05619597,  0.09895289],
       ...,
       [-0.08180201,  0.20603983, -0.05556207, ..., -0.02543889,
        -0.14692745,  0.01059252],
       [-0.347575  ,  0.22817251, -0.10940499, ..., -0.1519975 ,
         0.0327225 , -0.08738925],
       [-0.16678132,  0.09090873, -0.01011783, ..., -0.01703776,
        -0.06691939,  0.13167305]], dtype=float32)

In [29]:
X1, X_test, y, y_test = train_test_split(X, y1, test_size=0.2, random_state=0)

#Support Vector Classifier

In [30]:
lsvc=LinearSVC()
hyperparameters={'penalty':['l1','l2'],'C':(1,2,3,4)}
svc=GridSearchCV(lsvc,hyperparameters,n_jobs=-1,cv=5)
svc.fit(X1,y)
y_pred_svc = svc.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py", line 1185, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss, dual)
  File "/usr/loca

In [31]:
svc.best_params_

{'C': 1, 'penalty': 'l2'}

In [32]:
print(classification_report(y_test,y_pred_svc))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86        84
           1       0.79      0.88      0.83        66

    accuracy                           0.85       150
   macro avg       0.85      0.85      0.85       150
weighted avg       0.85      0.85      0.85       150



#Logistic Regression

In [33]:
LR=LogisticRegression(solver='liblinear')
hyperparameters={'penalty':['l1','l2'],'C':(1,2,3,4)}
lr=GridSearchCV(LR,hyperparameters,n_jobs=-1,cv=5)
lr.fit(X1,y)
y_pred_lr=lr.predict(X_test)#0.71

In [34]:
lr.best_params_

{'C': 2, 'penalty': 'l2'}

In [35]:
print(classification_report(y_test,y_pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.82      0.87        84
           1       0.80      0.91      0.85        66

    accuracy                           0.86       150
   macro avg       0.86      0.87      0.86       150
weighted avg       0.87      0.86      0.86       150



#Random Forest Classifier

In [36]:
# RFC = RandomForestClassifier(criterion='entropy', n_estimators=200).fit(X, y)
RFC= RandomForestClassifier()
hyperparameters={'n_estimators': [100,200,300,400,500],'criterion' :['gini', 'entropy']}
rfc=GridSearchCV(RFC,hyperparameters,n_jobs=-1,cv=5)
rfc.fit(X1,y)
y_pred_rfc=rfc.predict(X_test)#0.63

In [37]:
print(classification_report(y_test,y_pred_rfc))

              precision    recall  f1-score   support

           0       0.90      0.74      0.81        84
           1       0.73      0.89      0.80        66

    accuracy                           0.81       150
   macro avg       0.81      0.82      0.81       150
weighted avg       0.82      0.81      0.81       150



In [38]:
rfc.best_params_

{'criterion': 'entropy', 'n_estimators': 200}